<a href="https://colab.research.google.com/github/bibin001/Deep-Learning/blob/main/pima_diabetes_simple_feed_forward_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#dataset
#https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database
#diabetes.csv

In [2]:
import torch
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from torch.utils.data import Dataset, DataLoader

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
file_path = '/content/gdrive/My Drive/Colab Notebooks/Computer Vision/examples/diabetes.csv'
df_diabetes=pd.read_csv(file_path)
df_diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
X=df_diabetes.drop('Outcome', axis=1)
y=df_diabetes['Outcome']
#X=df_diabetes.iloc[:,:-1].values    #OR directly we can use these two line to output as array
#y=df_diabetes.iloc[:,-1].values
X=X.values
y=y.values


In [23]:
#Train Test and validation Split
ss=StratifiedShuffleSplit(n_splits=1, test_size=0.4, random_state=5)

for train_index, group_index in ss.split(X,y):
  xtrain, xgroup= X[train_index], X[group_index]
  ytrain, ygroup= y[train_index], y[group_index]

ss=StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=5)
for test_index, valid_index in ss.split(X,y):
  xtest, xvalid= X[test_index], X[valid_index]
  ytest, yvalid= y[test_index], y[valid_index]

In [24]:
#standardize the features or values X
sc=StandardScaler()
xtrain=sc.fit_transform(xtrain)
xtest=sc.transform(xtest)
xvalid=sc.transform(xvalid)

#convert to tensors
xtrain=torch.FloatTensor(xtrain)
xtest=torch.FloatTensor(xtest)
xvalid=torch.FloatTensor(xvalid)

ytrain, ytest, yvalid= map(torch.tensor, (ytrain, ytest, yvalid))

In [16]:
#Prepare to load the xtrain data in small chunks
class dds(Dataset):
  def __init__(self, x, y):
    self.x=x
    self.y=y

  def __getitem__(self, index):
    return self.x[index], self.y[index]

  def __len__(self):
    return len(self.x)

In [18]:
mydata=dds(xtrain, ytrain)
train_loader=DataLoader(dataset=mydata, batch_size=32)

In [19]:
#Create neural network architecture
class FeedForward(torch.nn.Module):
  def __init__(self,n_inputs,n_outputs,n_hidden_size):
    super(FeedForward,self).__init__()
    self.inputs=n_inputs
    self.outputs=n_outputs
    self.hidden=n_hidden_size

    #define the layers
    self.fc1=torch.nn.Linear(self.inputs,self.hidden)
    self.fc2=torch.nn.Linear(self.hidden,self.hidden)
    self.fc3=torch.nn.Linear(self.hidden,self.hidden)
    self.fc4=torch.nn.Linear(self.hidden,self.hidden)
    self.fc5=torch.nn.Linear(self.hidden,self.outputs)

    #define the activation function
    self.relu=torch.nn.ReLU()

    #initialize the weights
    self.init_weights()

  def init_weights(self):
    torch.nn.init.kaiming_normal_(self.fc1.weight)
    torch.nn.init.kaiming_normal_(self.fc2.weight)
    torch.nn.init.kaiming_normal_(self.fc3.weight)
    torch.nn.init.kaiming_normal_(self.fc4.weight)
    torch.nn.init.kaiming_normal_(self.fc5.weight)

  def forward(self,features):
    output=self.fc1(features)
    output=self.relu(output)
    output=self.fc2(output)
    output=self.relu(output)
    output=self.fc3(output)
    output=self.relu(output)
    output=self.fc4(output)
    output=self.relu(output)
    output=self.fc5(output)
    return output

In [40]:
#Create a network object
model=FeedForward(n_inputs=xtrain.shape[1], n_outputs=2, n_hidden_size=100)

#Create loss criteria
criteria= torch.nn.CrossEntropyLoss(reduction='mean')

#create an optimization or gradient
optimizer=torch.optim.SGD(params=model.parameters(), lr=.001, momentum=0.8)
#optimizer=torch.optim.Adam(params=model.parameters(), lr=.001)

In [39]:
#train the newtork
epochs=10
valid_list=[0,0]
patience=0

for epoch in range(epochs):
  for features, targets in train_loader:
    #forward propagation
    output= model.forward(features)

    #calculate the loss
    loss=criteria(output,targets)

    #initialize gradient to zero
    optimizer.zero_grad()

    ##backpropagation
    loss.backward()

    #update the weights
    optimizer.step()

  model.eval() #stop training

  c=torch.argmax(output.data, dim=1)
  train_accuracy= (c==targets).sum().item()/targets.shape[0]
  # print('Training Accuracy:', train_accuracy)

  #use model to predict validation set
  predict=model.forward(xvalid)
  c1=torch.argmax(xvalid.data, dim=1)
  valid_accuracy= (c1==yvalid).sum().item()/yvalid.shape[0]
  #valid_accuracy=0.63456
  print('Epoch {}/{}, Loss: {:.4f}, Train Accuracy: {:.4f}, Valid Accuracy: {:.4f}'.format(epoch,epochs,loss,train_accuracy,valid_accuracy))

  model.train()


  #Early stopping
  valid_list.append(valid_accuracy)

  if valid_list[-1] < valid_list[-2]:
    patience += 1
    if patience > 2:
      break

  if valid_list[-1] > valid_list[-2]:
    patience -= 1
    if patience < 0:
      patience=0

Epoch 0/10, Loss: 0.7642, Train Accuracy: 0.5000, Valid Accuracy: 0.1562
Epoch 1/10, Loss: 0.7387, Train Accuracy: 0.5000, Valid Accuracy: 0.1562
Epoch 2/10, Loss: 0.7216, Train Accuracy: 0.5000, Valid Accuracy: 0.1562
Epoch 3/10, Loss: 0.7091, Train Accuracy: 0.5000, Valid Accuracy: 0.1562
Epoch 4/10, Loss: 0.6989, Train Accuracy: 0.5000, Valid Accuracy: 0.1562
Epoch 5/10, Loss: 0.6900, Train Accuracy: 0.6667, Valid Accuracy: 0.1562
Epoch 6/10, Loss: 0.6819, Train Accuracy: 0.6667, Valid Accuracy: 0.1562
Epoch 7/10, Loss: 0.6740, Train Accuracy: 0.6667, Valid Accuracy: 0.1562
Epoch 8/10, Loss: 0.6661, Train Accuracy: 0.6667, Valid Accuracy: 0.1562
Epoch 9/10, Loss: 0.6586, Train Accuracy: 0.6667, Valid Accuracy: 0.1562


In [41]:
with torch.no_grad():
  predict = model.forward(xtest)
  c= torch.argmax(predict.data,dim=1)
  test_accuracy= (c1==ytest).sum().item()/ytest.shape[0]
  print('Testing Accuracy:', test_accuracy)

Testing Accuracy: 0.14322916666666666
